In [2]:
# importation des modules 
import glob
import os
import numpy as np 
import pandas as pd

from pathlib import Path

from PIL import Image, ImageStat
from skimage import io
import cv2


In [1]:
""" Avant de pouvoir exécuter le suite du code en interne il faut s'assurer de : 
    . avoir télécharger la base de donnée Kaggle :  https://www.kaggle.com/datasets/vbookshelf/v2-plant-seedlings-dataset
    . avoir modifier le nom du dossier de l'espèce ShepherdÔÇÖs Purse par Shepherds Purse
    . avoir supprimer le fichier : nonsegmentedv2
    . modifier la variable lien par le chemin où se trouve la base de donnée """
    
# SI IMPORTATION EN INTERNE LE LIENS DOIT ETRE CELUI OU LA BASE DE DONNEE EST STOCKEE :
lien = 'C:\\Users\\charl\\OneDrive\\Documents\\Formation\\Datascientest\\Projet\\Dataset'

In [3]:
# CREATION DU DATAFRAME

dossier_parent = Path(lien)

plants_name = [name for name in os.listdir(dossier_parent) if os.path.isdir(os.path.join(dossier_parent, name))]

#filepath des images et labels
filepaths = list(dossier_parent.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='filepath').astype(str)
labels = pd.Series(labels, name='species')

#Concatener filepaths et labels
df = pd.concat([filepaths, labels], axis=1)

#Mélange du DF et reset index
df = df.sample(frac=1).reset_index(drop = True)

#Fonction pour extraire le nom de l'image à partir du chemin du fichier
def extract_image_name(filepaths):
    return os.path.basename(filepaths)

#Ajout colonne image_name : Fonction à la colonne 'filepath' pour obtenir le nom de l'image
df['image_name'] = df['filepath'].apply(extract_image_name)

#Fonction pour obtenir la hauteur et la largeur de l'image
def get_image_size(filepath):
    with Image.open(filepath) as img:
        return img.size

#Ajoute colonne height et width : Fonction sur chaque filepath
df['height'] = df['filepath'].apply(lambda x: get_image_size(x)[1])
df['width'] = df['filepath'].apply(lambda x: get_image_size(x)[0])

#Ajout des colonnes des moyennes de couleurs pour chaque image
df['mean']= df["filepath"].apply(lambda x: ImageStat.Stat(Image.open(x)).mean)
df['R_mean'] = df['mean'].apply(lambda x : x[0])
df['G_mean'] = df['mean'].apply(lambda x : x[1])
df['B_mean'] = df['mean'].apply(lambda x : x[2])
df['luminosity'] = df['mean'].apply(lambda x : sum(x)/len(x))
df =df.drop(['mean'], axis =1)

# Fonction calculant la netteté de l'image avec Sobel 
def tenengrad(image):
    if image is None:
        print("Error: Could not read image at path: ", image)
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    G = np.sqrt(gx**2 + gy**2)
    T = np.sum(G)
    T = T/100000
    return T

# Ajout d'une colonne quantifiant le flou d'une image
df['sharpness_score'] = df["filepath"].apply(lambda x : tenengrad(cv2.imread(x)))

#Ajout colonne image_size : Calculer la taille (en pixels) de chaque image en multipliant la hauteur par la largeur
df['image_size'] = df['height'] * df['width']

#Ajout d'une colonne square
df["square"] = pd.Series("square" if (df['width'][i] == df['height'][i]) else "not square" for i in range(df.shape[0]))

print("Nombre d'images dans le dataset V2 Plant Seedlings Dataset : ", df.shape[0])

# Tri du DataFrame par espece 
df = df.sort_values("species", axis = 0)

#Sauvegarde du DataFrame
df.to_csv('V2_Plant_Seedlings_DataFrame.csv', index=False)

Nombre d'images dans le dataset V2 Plant Seedlings Dataset :  5539


In [4]:
df.head()

,filepath,species,image_name,height,width,R_mean,G_mean,B_mean,luminosity,sharpness_score,image_size,square
4258,C:\Users\charl\OneDrive\Documents\Formation\Da...,Black-grass,226.png,677,677,86.113840,66.805812,48.334554,67.084735,145.958276,458329,square
3802,C:\Users\charl\OneDrive\Documents\Formation\Da...,Black-grass,293.png,372,372,73.470864,60.303474,39.410987,57.728442,37.187340,138384,square
4858,C:\Users\charl\OneDrive\Documents\Formation\Da...,Black-grass,303.png,993,993,76.679368,68.112105,61.193974,68.661816,284.984739,986049,square
4870,C:\Users\charl\OneDrive\Documents\Formation\Da...,Black-grass,268.png,575,575,92.862639,80.840883,67.836416,80.513313,126.462691,330625,square
1523,C:\Users\charl\OneDrive\Documents\Formation\Da...,Black-grass,239.png,1571,1571,86.220835,67.749998,48.479208,67.483347,762.757020,2468041,square


•	filepath permet d’accéder à chaque image du jeu de donnée par le biais de son chemin d’accès vers le fichier correspondant.

•	species cumule 12 modalités correspondant à l’espèce de la plante représentée sur l’image en question.

•	image_name  est le nom du fichier.

•	height et width sont respectivement la hauteur et la taille de l’image.

•	image_size est le nombre de pixels composant l’image.

•	square possède 2 modalités et décrit si l’image est carrée ou non.

•	R_mean, V_mean et B_mean désignent la moyenne de l’intensité lumineuse d’un pixel de l’image dans le codage RVB, c’est-à-       dire l’intensité d’un pixel de l’image respectivement en rouge, vert et bleu.

•	luminosity quantifie la luminosité de l’image : plus la valeur est élevée, moins l’image  est sombre.

•	sharpness_score quantifie la netteté d’une image : plus la valeur est élevée, plus l’image est nette.
